 读取训练文件

In [1]:
f = open('链路预测数据集/FB15k/freebase_mtr100_mte100-train.txt')
data = f.read()
triples = data.split('\n')
triples = triples[:483142]

totals = list([entity_and_relation for triple in triples for entity_and_relation in triple.split('\t')])
print('三元组数量',len(triples))
print('实体总数',len(set(totals)))
#嵌入的总数为16296

三元组数量 483142
实体总数 16296


生成头实体集合 尾实体集合 关系集合

In [2]:
head_entities = list([triple.split('\t')[0] for triple in triples])
relations =  list([triple.split('\t')[1] for triple in triples])
tail_entities =  list([triple.split('\t')[2] for triple in triples])

total_entities = head_entities + tail_entities

print('实体数',len(set(head_entities + tail_entities)))
print('关系数',len(set(relations)))

实体数 14951
关系数 1345


生成负采样三元组

In [3]:
import random

# 生成负采样的三元组
invalid_head_entities = []
invalid_relations = []
invalid_tail_entities = []

for i in range(len(relations)):
    random_int = random.randint(0,len(head_entities)-1)
    random_int1 = random.randint(0,len(head_entities)-1)
    
    # 不同时替换头尾实体
    if i%2 ==0:
        invalid_head_entities.append(head_entities[i])
        invalid_relations.append(relations[i])
        invalid_tail_entities.append(tail_entities[random_int1])
    else:
        invalid_head_entities.append(head_entities[random_int])
        invalid_relations.append(relations[i])
        invalid_tail_entities.append(tail_entities[i])
#     invalid_head_entities.append(head_entities[random_int])
#     invalid_relations.append(relation)
#     invalid_tail_entities.append(tail_entities[random_int1])

print(head_entities[123456],relations[123456],tail_entities[123456])
print(head_entities[12345],relations[12345],tail_entities[12345])
print(head_entities[12234],relations[12234],tail_entities[12234])

print(invalid_head_entities[123456],invalid_relations[123456],invalid_tail_entities[123456])
print(invalid_head_entities[12345],invalid_relations[12345],invalid_tail_entities[12345])
print(invalid_head_entities[12234],invalid_relations[12234],invalid_tail_entities[12234])

/m/02w2bc /education/educational_institution/students_graduates./education/education/major_field_of_study /m/0g26h
/m/0kpzy /location/us_county/county_seat /m/0dc95
/m/081pw /military/military_conflict/combatants./military/military_combatant_group/combatants /m/0193qj
/m/02w2bc /education/educational_institution/students_graduates./education/education/major_field_of_study /m/0sxkh
/m/016mhd /location/us_county/county_seat /m/0dc95
/m/081pw /military/military_conflict/combatants./military/military_combatant_group/combatants /m/09nqf


对实体编码

In [4]:
# 为每个实体/关系分配倒排索引
# 根据 名称 查找 id
reverse_index = dict([key,index] for index,key in enumerate(set(totals)))
print('索引长度',len(reverse_index))

#数据标签化

head_entities_encode = list([reverse_index[entry] for entry in head_entities])
relations_encode = list([reverse_index[relation] for relation in relations])
tail_entities_encode = list([reverse_index[entry] for entry in tail_entities])

invalid_head_entities_encode = list([reverse_index[entry] for entry in invalid_head_entities])
invalid_relations_encode = list([reverse_index[relation] for relation in invalid_relations])
invalid_tail_entities_encode = list([reverse_index[entry] for entry in invalid_tail_entities])

print(head_entities_encode[123456],relations_encode[123456],tail_entities_encode[123456])
print(head_entities_encode[12345],relations_encode[12345],tail_entities_encode[12345])
print(head_entities_encode[12234],relations_encode[12234],tail_entities_encode[12234])

print(invalid_head_entities_encode[123456],invalid_relations_encode[123456],invalid_tail_entities_encode[123456])
print(invalid_head_entities_encode[12345],invalid_relations_encode[12345],invalid_tail_entities_encode[12345])
print(invalid_head_entities_encode[12234],invalid_relations_encode[12234],invalid_tail_entities_encode[12234])

索引长度 16296
5633 9134 12907
537 11752 4882
12520 4639 12228
5633 9134 10677
10621 11752 4882
12520 4639 1595


组合数据

In [5]:
# 编码后的数据
golden_data = list(zip(head_entities_encode,relations_encode,tail_entities_encode))
invalid_data = list(zip(invalid_head_entities_encode,invalid_relations_encode,invalid_tail_entities_encode))
    
print(golden_data[123456])
print(invalid_data[123456])

print('三元组数量',len(golden_data))
print('三元组数量',len(invalid_data))

(5633, 9134, 12907)
(5633, 9134, 10677)
三元组数量 483142
三元组数量 483142


获取数据

In [6]:
keep = 0 
def nextbatch(batchsize=400):
    global keep
    start = keep
    end = start + batchsize
    if end > len(golden_data):
        end = end % len(golden_data)
        keep = end
        return golden_data[start:]+golden_data[:end],invalid_data[start:]+invalid_data[:end]
    else:
        keep = end
        return golden_data[start:end],invalid_data[start:end]
    
# golden_data = golden_data[:10]
# invalid_data = invalid_data[:10]  

# print(nextbatch(3)[0])
# print(nextbatch(5)[0])
# print(nextbatch(4)[0])
# print(nextbatch(4)[0])
# print(nextbatch(4)[0])

In [ ]:
import tensorflow as tf
import math
import time

vocabulary_size = 16296
embedding_size = 50
epochs_num = 50
batch_size = 401
margin = 1
learning_rate = 1e-1

#定义模型
def l1_energy(batch):
    return tf.reduce_sum(tf.abs(batch[:, 0, :] + batch[:, 1, :] - batch[:, 2, :]), 1)

embedding_table = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size], stddev=1.0 / math.sqrt(embedding_size)))

pos_triples_batch = tf.placeholder(tf.int32, shape=[batch_size, 3])
neg_triples_batch = tf.placeholder(tf.int32, shape=[batch_size, 3])
predict_triples_batch = tf.placeholder(tf.int32,shape=[1,1])

pos_embed_batch = tf.nn.embedding_lookup(embedding_table, pos_triples_batch)
neg_embed_batch = tf.nn.embedding_lookup(embedding_table, neg_triples_batch)
predict_embed_batch = tf.nn.embedding_lookup(embedding_table, predict_triples_batch)

p_loss = l1_energy(pos_embed_batch)
n_loss = l1_energy(neg_embed_batch)

loss = tf.reduce_sum(tf.nn.relu(margin + p_loss - n_loss))
optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss)

sess = tf.Session()
sess.run(tf.initialize_all_variables())

for step in range(epochs_num):
    time_start = time.time() #开始计时
    loss_total = 0
    print('start:',keep)
    
    for i in range(len(golden_data)//batch_size + 1):
        
        golden_batch,invalid_batch = nextbatch(batchsize=batch_size)    
        feed_dict = {pos_triples_batch: golden_batch, neg_triples_batch: invalid_batch}
        loss_val, _ = sess.run([loss, optimizer], feed_dict=feed_dict)
        loss_total += loss_val
        
    print("\rstep %d, loss_val %f" % (step, loss_total/(len(golden_data)//batch_size + 1)))
    time_end = time.time()    #结束计时
    sum_t= time_end - time_start   #运行所花时间
    print('time cost', sum_t, 's')

Instructions for updating:
Use `tf.global_variables_initializer` instead.
start: 0
step 0, loss_val 185.383485
time cost 2.8773000240325928 s
start: 63
step 1, loss_val 23.419429
time cost 2.574111223220825 s
start: 126
step 2, loss_val 6.805881
time cost 2.6130075454711914 s
start: 189
step 3, loss_val 2.649368
time cost 3.07078218460083 s
start: 252
step 4, loss_val 1.293642
time cost 3.121647357940674 s
start: 315
step 5, loss_val 0.752976
time cost 2.5950543880462646 s
start: 378
step 6, loss_val 0.525403
time cost 3.6512303352355957 s
start: 441
step 7, loss_val 0.426328
time cost 3.451761484146118 s
start: 504
step 8, loss_val 0.370923
time cost 3.046847105026245 s
start: 567
step 9, loss_val 0.346636
time cost 2.748673915863037 s
start: 630
step 10, loss_val 0.325319
time cost 2.9361133575439453 s
start: 693
step 11, loss_val 0.315719
time cost 3.086737871170044 s
start: 756
step 12, loss_val 0.305632
time cost 3.0418596267700195 s
start: 819
step 13, loss_val 0.302563
time cost

In [ ]:
total_embedding_index = []

for entity in list(reverse_index):
    encode = reverse_index[entity]
    embedding = sess.run([predict_embed_batch], feed_dict= {predict_triples_batch: [[encode]]})[0][0][0]
    total_embedding_index.append([entity,embedding])
    
TransE_index = dict(total_embedding_index)
# print(TransE_index['/m/035yzw'])

entity_embedding_index = []
for entity in list(total_entities):
    encode = reverse_index[entity]
    embedding = sess.run([predict_embed_batch], feed_dict= {predict_triples_batch: [[encode]]})[0][0][0]
    entity_embedding_index.append([entity,embedding])
    
TransE_entity_index = dict(entity_embedding_index)

sess.close()

链接预测评测代码

In [ ]:
# 读入训练数据
# 先用hit@10标准测试一下
valid_file = open('链路预测数据集/FB15k/freebase_mtr100_mte100-valid.txt')
valid_data = valid_file.read()
valid_triples = valid_data.split('\n')[:50000]

#生成验证集实体和关系集合
valid_head_entities = list([triple.split('\t')[0] for triple in valid_triples])
valid_relations =  list([triple.split('\t')[1] for triple in valid_triples])
valid_tail_entities =  list([triple.split('\t')[2] for triple in valid_triples])

In [ ]:
print('开始预测')
import numpy as np
# import numpy_gpu as np
# import minpy.numpy as np

hit10 = 0
hit50 = 0
hit1000 = 0
num = 0

def takeFirst(elem):
    return elem[0]

# for head,relation,tail in zip(head_entities[:10],relations[:10],tail_entities[:10]):
for head,relation,tail in zip(valid_head_entities,valid_relations,valid_tail_entities):
    num = num+1
    
    print(head,relation,tail)
    #　头实体嵌入
    valid_head_embedding = TransE_index[head]
    #　关系的嵌入
    valid_relation_embedding = TransE_index[relation]
    #　头尾实体加和
    head_and_relation = valid_head_embedding + valid_relation_embedding
    # 预测一个尾实体
    predict_tail_embedding = head_and_relation
    
    #　与所有的实体计算距离
    distance_list = []
    
    # 遍历TransE嵌入中的所有实体
    for index,entity in enumerate(list(TransE_entity_index)):
        
        #　与TransE中的所有实体进行比较
        embedding = TransE_index[entity]
        
        # 二阶距离
        distance = np.linalg.norm(predict_tail_embedding - embedding)
#         distance = np.sqrt(np.sum(np.square(predict_tail_embedding - embedding)))
        distance_list.append([distance,entity])

    # 对所有距离进行排序
    distance_list.sort(key=takeFirst)

    print(distance_list[0])

    predict_1000_list = list([entity[1] for entity in distance_list])[:1000]
    predict_50_list = list([entity[1] for entity in distance_list])[:50]
    predict_10_list = list([entity[1] for entity in distance_list])[:10]

    if tail in predict_1000_list:
        hit1000 = hit1000+1
#         print('hit@1000yes')
#     else:
#         print('hit@1000no')
        
    if tail in predict_50_list:
        hit50 = hit50+1
#         print('hit@50yes')
#     else:
#         print('hit@50no')
        
    if tail in predict_10_list:
        hit10 = hit10 +1
#         print('hit@10yes')
#     else:
#         print('hit@10no')
        
    print('hit',hit10/num,hit50/num,hit1000/num,num)